In [ ]:
# @title Import Libraries
import sys
import math
import glob, os
import cv2
import numpy as np
import pandas as pd
import geopy.distance
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import exposure
from skimage import feature
import skimage.measure


In [ ]:
# Setup paths and import local libraries
%cd ../../

prj_path = Path('SkyAI-Sim')
sys.path.insert(0, str(prj_path))
from src.utils import img_helper
from src.utils.io_helper import str_to_floats
from src.utils import consts
from src.data.googlemap import GoogleMap

args = consts.ARGS
args

In [ ]:
aerial_data = GoogleMap(
        args=args,
        map_type='roadmap',
        data_dir=args.data_dir,
        overlap=args.overlap
        )
aerial_data.config(download_raster=True)


In [ ]:
aerial_imgs_path = glob.glob(os.path.join(aerial_data.data_dir / 'images', "*.jpg"))
aerial_imgs_path.sort()
print(len(aerial_imgs_path))
aerial_imgs, aerial_titles = img_helper.choose_random_images(nx*ny,
                                                            aerial_imgs_path,
                                                            output_dir=aerial_data.data_dir / 'random_sample',
                                                            obj=aerial_data,
                                                            seed=seed)

In [ ]:
import skimage.measure
entropys = [skimage.measure.shannon_entropy(img) for img in aerial_imgs]

img_helper.plot_multy(aerial_imgs, aerial_data.data_dir /'entropies', nx, ny, entropys)

In [ ]:
road_n = [cv2.normalize(img, None, alpha = 0, beta = 255,
                       norm_type = cv2.NORM_MINMAX,
                       dtype = cv2.CV_32F).astype('uint8') for img in aerial_imgs]
road_gray = [cv2.cvtColor(real_img, cv2.COLOR_BGR2GRAY) for real_img in road_n]
img_helper.plot_multy(road_gray, '../Gray Scale \nRoadmap Images', nx, ny, aerial_titles)

In [ ]:

road_eq = [cv2.equalizeHist(img) for img in road_gray]
img_helper.plot_multy(road_eq, '../Equalized Gray Scale \nRoadmap Images', nx, ny, aerial_titles)

In [ ]:
# equalize each color channel
eq_color = [cv2.merge([cv2.equalizeHist(channel) for channel in cv2.split(img)]) for img in road_n]
img_helper.plot_multy(eq_color, '../Equalized Color \nRoadmap Images', nx, ny, aerial_titles)